In [48]:
import numpy as np
from sklearn import preprocessing 
import pandas as pd


data = pd.read_csv('life-expectancy.csv') 
data

,Country Name,Country Code,Region,IncomeGroup,Year,Life Expectancy World Bank,Prevelance of Undernourishment,CO2,Health Expenditure %,Education Expenditure %,Unemployment,Corruption,Sanitation,Injuries,Communicable,NonCommunicable
0,Afghanistan,AFG,South Asia,Low income,2001,56.308,47.8,730.000000,NaN,NaN,10.809000,NaN,NaN,2179727.10,9689193.70,5795426.38
1,Angola,AGO,Sub-Saharan Africa,Lower middle income,2001,47.059,67.5,15960.000000,4.483516,NaN,4.004000,NaN,NaN,1392080.71,11190210.53,2663516.34
2,Albania,ALB,Europe & Central Asia,Upper middle income,2001,74.288,4.9,3230.000000,7.139524,3.45870,18.575001,NaN,40.520895,117081.67,140894.78,532324.75
3,Andorra,AND,Europe & Central Asia,High income,2001,NaN,NaN,520.000000,5.865939,NaN,NaN,NaN,21.788660,1697.99,695.56,13636.64
4,United Arab Emirates,ARE,Middle East & North Africa,High income,2001,74.544,2.8,97200.000000,2.484370,NaN,2.493000,NaN,NaN,144678.14,65271.91,481740.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3301,Vanuatu,VUT,East Asia & Pacific,Lower middle income,2019,70.474,12.4,209.999993,3.360347,1.77788,1.801000,3.0,NaN,12484.18,26032.56,69213.56
3302,Samoa,WSM,East Asia & Pacific,Lower middle income,2019,73.321,4.4,300.000012,6.363094,4.70625,8.406000,4.0,47.698788,6652.84,9095.19,43798.62
3303,South Africa,ZAF,Sub-Saharan Africa,Upper middle income,2019,64.131,6.3,439640.014648,9.109355,5.91771,28.469999,NaN,NaN,3174676.10,13198944.71,10214261.89
3304,Zambia,ZMB,Sub-Saharan Africa,Low income,2019,63.886,NaN,6800.000191,5.312203,4.46518,12.520000,2.5,NaN,510982.75,4837094.00,2649687.82


life-expectancy

In [49]:
raw_csv_data = np.genfromtxt('life-expectancy.csv', delimiter =',',filling_values=np.nan)

df = pd.read_csv('life-expectancy.csv')

Dealing with NULL values

In [50]:
for i in range(raw_csv_data.shape[1]):
    if np.any(np.isnan(raw_csv_data[:,i])) == True:
        print(f"Null value in cloumn {i+1}")

Null value in cloumn 1
Null value in cloumn 2
Null value in cloumn 3
Null value in cloumn 4
Null value in cloumn 5
Null value in cloumn 6
Null value in cloumn 7
Null value in cloumn 8
Null value in cloumn 9
Null value in cloumn 10
Null value in cloumn 11
Null value in cloumn 12
Null value in cloumn 13
Null value in cloumn 14
Null value in cloumn 15
Null value in cloumn 16


In [51]:
mean_review = np.nanmean(raw_csv_data[:,6])
nan_indices_review = np.isnan(raw_csv_data[:,6]) 
raw_csv_data[nan_indices_review,6]=mean_review

raw_csv_data=raw_csv_data[~(np.any(np.isnan(raw_csv_data),axis =1))]

Dealing with duplicates

In [52]:
raw_csv_data = np.unique(raw_csv_data,axis =0)
unscaled_inputs_all = raw_csv_data[:,1:-1] # 1-> second column till -1 -> last column excluded
targets_all = raw_csv_data[:,-1]

In [53]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0]) 
seed =2000 
np.random.seed(seed) 
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices] 
targets_all = targets_all[shuffled_indices]


Balance the data

In [54]:
count_of_1s= np.sum(targets_all == 1) 
count_of_0s = 0 # currently 0
indices_to_remove = [] # we want it to be list or tuple hence we put empty brackets

for i in range(targets_all.shape[0]): # shape on the zero axis is the length of the vector	if targets_all[i]==0:	count_of_0s +=1	
    if targets_all[i]==0:
        count_of_0s += 1
        if count_of_0s > count_of_1s:
            indices_to_remove.append(i)
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all,indices_to_remove,axis = 0)
targets_equal_priors = np.delete(targets_all,indices_to_remove,axis=0)


Standardize the inputs

In [55]:
if unscaled_inputs_equal_priors.shape[0] > 0:
    scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)
    # When the data was collected it was actually arranged by date
    # Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
    # Since we will be batching, we want the data to be as randomly spread out as possible
    shuffled_indices = np.arange(scaled_inputs.shape[0]) 
    np.random.seed(seed) 
    np.random.shuffle(shuffled_indices)

    # Use the shuffled indices to shuffle the inputs and targets. 
    shuffled_inputs = scaled_inputs[shuffled_indices] 
    shuffled_targets = targets_equal_priors[shuffled_indices]
else:
    print("Error: input array has 0 samples")


Error: input array has 0 samples


Split the data into training, validation and test

In [56]:
samples_count = shuffled_inputs.shape[0]
train_count = int(0.8 * samples_count)
validation_count = int(0.1 * samples_count)
test_count = samples_count - train_count - validation_count
train_inputs = shuffled_inputs[:train_count]
train_targets = shuffled_targets[:train_count]

In [57]:
validation_inputs = shuffled_inputs[train_count:train_count+validation_count]
validation_targets = shuffled_targets[train_count:train_count+validation_count]
test_inputs = shuffled_inputs[train_count+validation_count:]
test_targets = shuffled_targets[train_count+validation_count:]
print(np.sum(train_targets),train_count,np.sum(train_targets)/train_count)
print(np.sum(validation_targets),validation_count,np.sum(validation_targets)/validation_count)
print(np.sum(test_targets),test_count,np.sum(test_targets)/test_count)

0.0 0 nan
0.0 0 nan
0.0 0 nan


C:\Users\prathamesh chougale\AppData\Local\Temp\ipykernel_3316\531385094.py:5: RuntimeWarning: invalid value encountered in scalar divide
  print(np.sum(train_targets),train_count,np.sum(train_targets)/train_count)
C:\Users\prathamesh chougale\AppData\Local\Temp\ipykernel_3316\531385094.py:6: RuntimeWarning: invalid value encountered in scalar divide
  print(np.sum(validation_targets),validation_count,np.sum(validation_targets)/validation_count)
C:\Users\prathamesh chougale\AppData\Local\Temp\ipykernel_3316\531385094.py:7: RuntimeWarning: invalid value encountered in scalar divide
  print(np.sum(test_targets),test_count,np.sum(test_targets)/test_count)


Now we save our data into npz file

In [58]:
np.savez('Audiobook_train_data',inputs = train_inputs,targets = train_targets)
np.savez('Audiobook_validation_data',inputs = validation_inputs,targets = validation_targets)
np.savez('Audiobook_test_data',inputs = test_inputs,targets = test_targets)